In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from scnn_model import SCNN_MODEL
from tensorflow.contrib import learn
from six.moves import cPickle as pickle

In [3]:
# Load vocabulary and the word2vec model
pickle_file = 'save.pickle'
with open(pickle_file, 'rb') as f :
    save = pickle.load(f)
    wordsVectors = save['wordsVectors']
    vocabulary = save['vocabulary']
    del save  # hint to help gc free up memory
print('Vocabulary and the word2vec loaded')
print('Vocabulary size is ', len(vocabulary))
print('Word2Vec model shape is ', wordsVectors.shape)

#Load training data, training labels, validation data, validation labels
pickle_file = 'data_saved.pickle'
with open(pickle_file, 'rb') as f :
    save = pickle.load(f)
    train_data = save['train_data']
    train_labels = save['train_labels']
    validation_data = save['validation_data']
    validation_labels = save['validation_labels']
    del save  # hint to help gc free up memory
print('train data shape ', train_data.shape)
print('train labels shape ', train_labels.shape)
print('validation data shape ', validation_data.shape)
print('validation labels shape ', validation_labels.shape)

Vocabulary and the word2vec loaded
Vocabulary size is  9687
Word2Vec model shape is  (9687, 300)
train data shape  (1280, 160)
train labels shape  (1280, 2)
validation data shape  (160, 160)
validation labels shape  (160, 2)


In [4]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    Original taken from https://github.com/dennybritz/cnn-text-classification-tf/blob/master/data_helpers.py
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [5]:
# Model Hyperparameters
SENTENCE_PER_REVIEW = 16
WORDS_PER_SENTENCE = 10
EMBEDDING_DIM = 300
FILTER_WIDTHS_SENT_CONV = np.array([3, 4, 5])
NUM_FILTERS_SENT_CONV = 100
FILTER_WIDTHS_DOC_CONV = np.array([3, 4, 5])
NUM_FILTERS_DOC_CONV = 100
NUM_CLASSES = 2
DROPOUT_KEEP_PROB = 0.5
L2_REG_LAMBDA = 0.0
BATCH_SIZE = 64
NUM_EPOCHS = 100
EVALUATE_EVERY = 100   # Evaluate model on the validation set after 100 steps
CHECKPOINT_EVERY = 100 # Save model after each 200 steps
NUM_CHECKPOINTS = 5    # Keep only the 5 most recents checkpoints
LEARNING_RATE = 1e-3   # The learning rate

In [6]:
'''Training the model'''
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        cnn = SCNN_MODEL(sentence_per_review=SENTENCE_PER_REVIEW, 
                        words_per_sentence=WORDS_PER_SENTENCE, 
                        wordVectors=wordsVectors, 
                        embedding_size=EMBEDDING_DIM, 
                        filter_widths_sent_conv=FILTER_WIDTHS_SENT_CONV, 
                        num_filters_sent_conv=NUM_FILTERS_SENT_CONV, 
                        filter_widths_doc_conv=FILTER_WIDTHS_DOC_CONV, 
                        num_filters_doc_conv=NUM_FILTERS_DOC_CONV, 
                        num_classes=NUM_CLASSES, 
                        l2_reg_lambda=L2_REG_LAMBDA)
        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))
        
        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
        
        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)
        
        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
        
        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=NUM_CHECKPOINTS)
        
        # Initialize all variables
        sess.run(tf.global_variables_initializer())
        
        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.input_size: BATCH_SIZE,
                cnn.dropout: DROPOUT_KEEP_PROB
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)
        
        
        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.input_size: y_batch.shape[0],
                cnn.dropout: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
        
        print('train data shape ', train_data.shape)
        print('train labels shape ', train_labels.shape)
        print('validation data shape ', validation_data.shape)
        print('validation labels shape ', validation_labels.shape)
        
        # Generate batches
        batches = batch_iter(
            list(zip(train_data, train_labels)), BATCH_SIZE, NUM_EPOCHS)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % EVALUATE_EVERY == 0:
                print("\nEvaluation:")
                dev_step(validation_data, validation_labels, writer=dev_summary_writer)
                print("")
            if current_step % CHECKPOINT_EVERY == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


Writing to /Users/MacBook/Documents/MLTraining/DECEPTIVE_REVIEWS_ON_HOTEL/runs/1503225456

train data shape  (1280, 160)
train labels shape  (1280, 2)
validation data shape  (160, 160)
validation labels shape  (160, 2)
2017-08-20T18:37:42.222602: step 1, loss 1.35769, acc 0.4375
2017-08-20T18:37:42.899740: step 2, loss 1.0918, acc 0.515625
2017-08-20T18:37:43.576323: step 3, loss 1.2094, acc 0.453125
2017-08-20T18:37:44.254820: step 4, loss 1.04435, acc 0.5
2017-08-20T18:37:44.926240: step 5, loss 0.903324, acc 0.5625
2017-08-20T18:37:45.601660: step 6, loss 0.908241, acc 0.5625
2017-08-20T18:37:46.280138: step 7, loss 1.18932, acc 0.4375
2017-08-20T18:37:46.959061: step 8, loss 1.04677, acc 0.4375
2017-08-20T18:37:47.635470: step 9, loss 1.09782, acc 0.484375
2017-08-20T18:37:48.343104: step 10, loss 0.901628, acc 0.46875
2017-08-20T18:37:49.020608: step 11, loss 1.08132, acc 0.4375
2017-08-20T18:37:49.694891: step 12, loss 1.28953, acc 0.34375
2017-08-20T18:37:50.365373: step 13, los

2017-08-20T18:39:10.357829: step 124, loss 0.645428, acc 0.640625
2017-08-20T18:39:11.115733: step 125, loss 0.843553, acc 0.53125
2017-08-20T18:39:11.859675: step 126, loss 0.806943, acc 0.578125
2017-08-20T18:39:12.640061: step 127, loss 0.889303, acc 0.46875
2017-08-20T18:39:13.400613: step 128, loss 0.787253, acc 0.578125
2017-08-20T18:39:14.157079: step 129, loss 0.7963, acc 0.5625
2017-08-20T18:39:14.914234: step 130, loss 0.804121, acc 0.578125
2017-08-20T18:39:15.670020: step 131, loss 0.783023, acc 0.515625
2017-08-20T18:39:16.430064: step 132, loss 0.836354, acc 0.515625
2017-08-20T18:39:17.199890: step 133, loss 0.763662, acc 0.546875
2017-08-20T18:39:17.978529: step 134, loss 0.901787, acc 0.546875
2017-08-20T18:39:18.735282: step 135, loss 0.825675, acc 0.625
2017-08-20T18:39:19.493107: step 136, loss 0.863815, acc 0.53125
2017-08-20T18:39:20.253134: step 137, loss 0.621518, acc 0.578125
2017-08-20T18:39:21.008645: step 138, loss 0.748317, acc 0.546875
2017-08-20T18:39:21.

2017-08-20T18:40:47.391083: step 247, loss 0.647145, acc 0.59375
2017-08-20T18:40:48.146778: step 248, loss 0.576275, acc 0.71875
2017-08-20T18:40:48.905739: step 249, loss 0.61807, acc 0.640625
2017-08-20T18:40:49.635200: step 250, loss 0.577706, acc 0.71875
2017-08-20T18:40:50.373613: step 251, loss 0.642458, acc 0.640625
2017-08-20T18:40:51.138558: step 252, loss 0.640839, acc 0.6875
2017-08-20T18:40:51.883255: step 253, loss 0.813531, acc 0.53125
2017-08-20T18:40:52.655351: step 254, loss 0.573277, acc 0.703125
2017-08-20T18:40:53.414869: step 255, loss 0.610387, acc 0.671875
2017-08-20T18:40:54.149059: step 256, loss 0.68096, acc 0.625
2017-08-20T18:40:54.885641: step 257, loss 0.715857, acc 0.578125
2017-08-20T18:40:55.619081: step 258, loss 0.466738, acc 0.8125
2017-08-20T18:40:56.354132: step 259, loss 0.62263, acc 0.65625
2017-08-20T18:40:57.098956: step 260, loss 0.716728, acc 0.609375
2017-08-20T18:40:57.890714: step 261, loss 0.633583, acc 0.6875
2017-08-20T18:40:58.652009:

2017-08-20T18:42:25.642648: step 370, loss 0.468158, acc 0.75
2017-08-20T18:42:26.397129: step 371, loss 0.393424, acc 0.828125
2017-08-20T18:42:27.173029: step 372, loss 0.435128, acc 0.8125
2017-08-20T18:42:27.946123: step 373, loss 0.511491, acc 0.765625
2017-08-20T18:42:28.703295: step 374, loss 0.415155, acc 0.765625
2017-08-20T18:42:29.459407: step 375, loss 0.535138, acc 0.75
2017-08-20T18:42:30.222472: step 376, loss 0.426116, acc 0.71875
2017-08-20T18:42:30.984314: step 377, loss 0.331433, acc 0.859375
2017-08-20T18:42:31.766509: step 378, loss 0.378068, acc 0.796875
2017-08-20T18:42:32.535137: step 379, loss 0.398239, acc 0.828125
2017-08-20T18:42:33.310849: step 380, loss 0.341832, acc 0.828125
2017-08-20T18:42:34.053887: step 381, loss 0.450231, acc 0.765625
2017-08-20T18:42:34.791974: step 382, loss 0.362856, acc 0.828125
2017-08-20T18:42:35.549960: step 383, loss 0.352904, acc 0.859375
2017-08-20T18:42:36.309882: step 384, loss 0.405724, acc 0.8125
2017-08-20T18:42:37.075

2017-08-20T18:44:04.269780: step 493, loss 0.255633, acc 0.875
2017-08-20T18:44:05.030860: step 494, loss 0.465637, acc 0.84375
2017-08-20T18:44:05.791973: step 495, loss 0.323183, acc 0.890625
2017-08-20T18:44:06.549374: step 496, loss 0.385077, acc 0.8125
2017-08-20T18:44:07.325521: step 497, loss 0.239273, acc 0.90625
2017-08-20T18:44:08.093897: step 498, loss 0.29826, acc 0.90625
2017-08-20T18:44:08.867951: step 499, loss 0.281104, acc 0.859375
2017-08-20T18:44:09.628757: step 500, loss 0.248125, acc 0.9375

Evaluation:
2017-08-20T18:44:10.162277: step 500, loss 0.424765, acc 0.80625

Saved model checkpoint to /Users/MacBook/Documents/MLTraining/DECEPTIVE_REVIEWS_ON_HOTEL/runs/1503225456/checkpoints/model-500

2017-08-20T18:44:12.018797: step 501, loss 0.158516, acc 0.953125
2017-08-20T18:44:12.722758: step 502, loss 0.194048, acc 0.9375
2017-08-20T18:44:13.458290: step 503, loss 0.289487, acc 0.84375
2017-08-20T18:44:14.224327: step 504, loss 0.201843, acc 0.921875
2017-08-20T18:4

...

2017-08-20T18:55:54.536670: step 1385, loss 0.000811192, acc 1
2017-08-20T18:55:55.295881: step 1386, loss 0.00321942, acc 1
2017-08-20T18:55:56.053489: step 1387, loss 0.00219668, acc 1
2017-08-20T18:55:56.840168: step 1388, loss 0.0028558, acc 1
2017-08-20T18:55:57.619317: step 1389, loss 0.00218255, acc 1
2017-08-20T18:55:58.369788: step 1390, loss 0.0021697, acc 1
2017-08-20T18:55:59.134199: step 1391, loss 0.00365676, acc 1
2017-08-20T18:56:00.000095: step 1392, loss 0.00241709, acc 1
2017-08-20T18:56:00.878242: step 1393, loss 0.00111142, acc 1
2017-08-20T18:56:01.806480: step 1394, loss 0.00380497, acc 1
2017-08-20T18:56:02.608690: step 1395, loss 0.00218216, acc 1
2017-08-20T18:56:03.418560: step 1396, loss 0.00142082, acc 1
2017-08-20T18:56:04.248471: step 1397, loss 0.00218851, acc 1
2017-08-20T18:56:05.047528: step 1398, loss 0.00172943, acc 1
2017-08-20T18:56:05.831836: step 1399, loss 0.00631842, acc 1
2017-08-20T18:56:06.609931: step 1400, loss 0.00572796, acc 1

Evaluati

2017-08-20T18:57:41.333802: step 1511, loss 0.000608376, acc 1
2017-08-20T18:57:42.182212: step 1512, loss 0.00161126, acc 1
2017-08-20T18:57:43.070188: step 1513, loss 0.00134117, acc 1
2017-08-20T18:57:44.098550: step 1514, loss 0.00176128, acc 1
2017-08-20T18:57:44.962774: step 1515, loss 0.0132984, acc 0.984375
2017-08-20T18:57:45.820111: step 1516, loss 0.00418385, acc 1
2017-08-20T18:57:46.627675: step 1517, loss 0.0179951, acc 0.984375
2017-08-20T18:57:47.402428: step 1518, loss 0.00137269, acc 1
2017-08-20T18:57:48.183722: step 1519, loss 0.000979603, acc 1
2017-08-20T18:57:48.977258: step 1520, loss 0.00681503, acc 1
2017-08-20T18:57:49.781538: step 1521, loss 0.00152694, acc 1
2017-08-20T18:57:50.590744: step 1522, loss 0.00448216, acc 1
2017-08-20T18:57:51.383060: step 1523, loss 0.00379335, acc 1
2017-08-20T18:57:52.180791: step 1524, loss 0.000744827, acc 1
2017-08-20T18:57:52.978118: step 1525, loss 0.00248843, acc 1
2017-08-20T18:57:53.800065: step 1526, loss 0.00789299,

2017-08-20T18:59:24.350297: step 1639, loss 0.00212378, acc 1
2017-08-20T18:59:25.104464: step 1640, loss 0.00162009, acc 1
2017-08-20T18:59:25.861069: step 1641, loss 0.00510614, acc 1
2017-08-20T18:59:26.631974: step 1642, loss 0.00257604, acc 1
2017-08-20T18:59:27.414278: step 1643, loss 0.00570138, acc 1
2017-08-20T18:59:28.170306: step 1644, loss 0.0313119, acc 0.984375
2017-08-20T18:59:28.908529: step 1645, loss 0.00136177, acc 1
2017-08-20T18:59:29.661130: step 1646, loss 0.020798, acc 0.984375
2017-08-20T18:59:30.417078: step 1647, loss 0.00975088, acc 1
2017-08-20T18:59:31.174857: step 1648, loss 0.0142672, acc 1
2017-08-20T18:59:31.933609: step 1649, loss 0.00406286, acc 1
2017-08-20T18:59:32.718089: step 1650, loss 0.00271973, acc 1
2017-08-20T18:59:33.482177: step 1651, loss 0.0158457, acc 1
2017-08-20T18:59:34.239364: step 1652, loss 0.0426655, acc 0.984375
2017-08-20T18:59:34.997607: step 1653, loss 0.0108189, acc 1
2017-08-20T18:59:35.757550: step 1654, loss 0.0316218, a

2017-08-20T19:01:04.756834: step 1766, loss 0.0377833, acc 0.984375
2017-08-20T19:01:05.520354: step 1767, loss 0.00800391, acc 1
2017-08-20T19:01:06.280611: step 1768, loss 0.0235765, acc 1
2017-08-20T19:01:07.040660: step 1769, loss 0.00912287, acc 1
2017-08-20T19:01:07.798318: step 1770, loss 0.0151833, acc 1
2017-08-20T19:01:08.572313: step 1771, loss 0.00851008, acc 1
2017-08-20T19:01:09.336892: step 1772, loss 0.00946524, acc 1
2017-08-20T19:01:10.092469: step 1773, loss 0.00966444, acc 1
2017-08-20T19:01:10.865596: step 1774, loss 0.0411162, acc 0.984375
2017-08-20T19:01:11.627075: step 1775, loss 0.0510406, acc 0.984375
2017-08-20T19:01:12.415922: step 1776, loss 0.00277172, acc 1
2017-08-20T19:01:13.176431: step 1777, loss 0.0209932, acc 0.984375
2017-08-20T19:01:13.929961: step 1778, loss 0.0138006, acc 1
2017-08-20T19:01:14.685509: step 1779, loss 0.026815, acc 0.984375
2017-08-20T19:01:15.450756: step 1780, loss 0.00790081, acc 1
2017-08-20T19:01:16.208628: step 1781, loss 

2017-08-20T19:02:45.627176: step 1893, loss 0.0292811, acc 1
2017-08-20T19:02:46.382074: step 1894, loss 0.00217327, acc 1
2017-08-20T19:02:47.151084: step 1895, loss 0.00416333, acc 1
2017-08-20T19:02:47.932753: step 1896, loss 0.00293034, acc 1
2017-08-20T19:02:48.689316: step 1897, loss 0.00985391, acc 1
2017-08-20T19:02:49.446371: step 1898, loss 0.00755094, acc 1
2017-08-20T19:02:50.206105: step 1899, loss 0.0204255, acc 1
2017-08-20T19:02:50.998038: step 1900, loss 0.00146715, acc 1

Evaluation:
2017-08-20T19:02:51.567091: step 1900, loss 0.638949, acc 0.8375

Saved model checkpoint to /Users/MacBook/Documents/MLTraining/DECEPTIVE_REVIEWS_ON_HOTEL/runs/1503225456/checkpoints/model-1900

2017-08-20T19:02:53.551788: step 1901, loss 0.00132592, acc 1
2017-08-20T19:02:54.238732: step 1902, loss 0.00692677, acc 1
2017-08-20T19:02:54.977579: step 1903, loss 0.00771842, acc 1
2017-08-20T19:02:55.739598: step 1904, loss 0.0193574, acc 0.984375
2017-08-20T19:02:56.509013: step 1905, loss 